## 기술 세트(Skillset)

기술 세트(Skillset)는 [인덱서(Indexer)](https://learn.microsoft.com/ko-kr/azure/search/search-indexer-overview)에 연결된 Azure AI 검색의 재사용 가능한 개체입니다. 여기에는 외부 데이터 원본에서 검색된 문서에 대해 기본 제공 AI 또는 외부 사용자 지정 처리를 호출하는 하나 이상의 기술이 포함되어 있습니다.

예를 들어, 이미지 또는 비정형 텍스트가 포함된 문서에서 텍스트 콘텐츠 및 구조를 생성하는 작업을 정의하고, 이미지에 대한 OCR, 개체 인식이 되었으나 특정할 수 없는 텍스트 번역등을 수행 할 수 있습니다. 기술 세트는 외부 데이터 원본에서 텍스트 및 이미지를 추출한 후 필드 매핑이 처리된 이후에 실행됩니다.

![Skillset process diagram](https://learn.microsoft.com/ko-kr/azure/search/media/cognitive-search-working-with-skillsets/skillset-process-diagram-1.png)

기술(Skill)은 Microsoft의 [기본 제공 기술](https://learn.microsoft.com/ko-kr/azure/search/cognitive-search-predefined-skills)이거나 외부에서 호스트하는 처리 논리에 대한 [사용자 지정 기술](https://learn.microsoft.com/ko-kr/azure/search/cognitive-search-create-custom-skill-example)일 수 있습니다. 기술 세트는 인덱싱 중에 사용되거나 지식 저장소에 프로젝션되는 보강된 문서를 생성합니다.

기술 세트 정의에 대한 규칙은 다음과 같습니다.

- 기술 세트 컬렉션 내의 고유한 이름. 기술 세트는 모든 인덱서에서 사용할 수 있는 최상위 리소스입니다.
- 하나 이상의 기술. 3~5개의 기술이 일반적입니다. 최댓값은 30입니다.
- 기술 세트는 동일한 유형의 기술(예: 여러 Shaper 기술)을 반복할 수 있습니다.
- 기술 세트는 연결된 작업, 루핑 및 분기를 지원합니다.
- 인덱서는 기술 세트 실행을 주도합니다. 기술 세트를 테스트하기 전에 인덱서, 데이터 원본 및 인덱스가 필요합니다.

> 📝 참고
>
> 더 자세한 내용은 [Azure AI 검색에서의 기술 세트 개념](https://learn.microsoft.com/ko-kr/azure/search/cognitive-search-working-with-skillsets), [Azure AI 검색에서 기술 세트 만들기](https://learn.microsoft.com/ko-kr/azure/search/cognitive-search-defining-skillset) 을 참고해 주세요.

## 텍스트 분할 예제

이 예제에서는 [문서 분할(Text Split)](https://learn.microsoft.com/azure/search/cognitive-search-skill-textsplit)을 통해 데이터 청킹(data chunking) 하는 과정을 보여줍니다.
>문서 분할은 문서를 특정 길이의 더 작은 덩어리로 잘라내는 것으로(예를 들어 10,000자의 문서를 100 혹은 200자 단위로 잘라내는 행위), 벡터 검색에서 자주 사용하는 방법입니다. 조금 더 자세한 내용은 [Azure AI 검색에서 벡터 검색 솔루션에 대한 대용량 문서 청크](https://learn.microsoft.com/ko-kr/azure/search/vector-search-how-to-chunk-documents)를 참고하세요.

[AzureOpenAIEmbedding](https://learn.microsoft.com/azure/search/cognitive-search-skill-azure-openai-embedding)은 사용자가 환경 변수에 제공한 연결 정보를 사용하여 Azure OpenAI에 대한 호출을 처리합니다. [인덱서 프로젝션(Indexer projection)](https://learn.microsoft.com/ko-kr/azure/search/search-how-to-define-index-projections)은 청크된 데이터에 사용되는 보조 인덱스를 지정합니다.

In [64]:
import os
import json
from dotenv import load_dotenv
from openai import AzureOpenAI
from azure.identity import DefaultAzureCredential, get_bearer_token_provider
from azure.core.credentials import AzureKeyCredential
from azure.search.documents.indexes import SearchIndexClient, SearchIndexerClient
from azure.search.documents import SearchClient
from azure.search.documents.indexes.models import (
    SimpleField,
    SearchFieldDataType,
    SearchableField,
    SearchField,
    VectorSearch,
    HnswAlgorithmConfiguration,
    VectorSearchProfile,
    SearchIndexer,
    SearchIndex,
    AzureOpenAIVectorizer,
    AzureOpenAIParameters,
    SearchIndexerDataSourceConnection,
    SearchIndexerDataContainer,
    SearchIndexerSkillset,
    InputFieldMappingEntry,
    OutputFieldMappingEntry,
    SplitSkill,
    AzureOpenAIEmbeddingSkill,
    SearchIndexerIndexProjections,
    SearchIndexerIndexProjectionSelector,
    SearchIndexerIndexProjectionsParameters,
    TextSplitMode
)

load_dotenv(override=True)

endpoint = os.getenv("AZURE_SEARCH_SERVICE_ENDPOINT")
credential = AzureKeyCredential(os.getenv("AZURE_SEARCH_ADMIN_KEY", "")) if len(os.getenv("AZURE_SEARCH_ADMIN_KEY", "")) > 0 else DefaultAzureCredential()

search_datasource = os.environ["AZURE_SEARCH_DATASOURCE"]

azure_openai_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
azure_openai_key = os.getenv("AZURE_OPENAI_KEY", "") if len(os.getenv("AZURE_OPENAI_KEY", "")) > 0 else None
azure_openai_embedding_deployment = os.getenv("AZURE_OPENAI_EMBEDDING_DEPLOYMENT", "text-embedding-ada-002")
azure_openai_embedding_dimensions = int(os.getenv("AZURE_OPENAI_EMBEDDING_DIMENSIONS", 1536))
azure_openai_api_version = os.getenv("AZURE_OPENAI_API_VERSION", "2024-06-01")

index_name = "skillset-split-text-test"

index_client = SearchIndexClient(endpoint=endpoint, credential=credential)

### skillset 용 index 생성

In [65]:
# index가 존재하면 삭제
try:
    index_client.get_index(index_name)
    index_client.delete_index(index_name)
    print(f"'{index_name}' 인덱스 삭제.")
except Exception as e:
    print(f"'{index_name}' 인덱스가 존재하지 않거나 삭제가 되지 않았습니다.: {e}")

'skillset-split-text-test' 인덱스 삭제.


In [84]:
# 스키마 
fields=[
    SearchField(
        name="chunk_id",
        type=SearchFieldDataType.String,
        key=True,
        hidden=False,
        filterable=True,
        sortable=True,
        facetable=False,
        searchable=True,
        analyzer_name="keyword"
    ),
    SearchField(
        name="parent_id",
        type=SearchFieldDataType.String,
        hidden=False,
        filterable=True,
        sortable=True,
        facetable=False,
        searchable=True
    ),
    SearchField(
        name="chunk",
        type=SearchFieldDataType.String,
        hidden=False,
        filterable=False,
        sortable=False,
        facetable=False,
        searchable=True
    ),
    SearchField(
        name="title",
        type=SearchFieldDataType.String,
        hidden=False,
        filterable=False,
        sortable=False,
        facetable=False,
        searchable=True
    ),
    SearchField(
        name="vector",
        type=SearchFieldDataType.Collection(SearchFieldDataType.Single),
        hidden=False,
        filterable=False,
        sortable=False,
        facetable=False,
        searchable=True,
        vector_search_dimensions=azure_openai_embedding_dimensions,
        vector_search_profile_name="profile"
    )
]

# 벡터 검색 구성
vector_search = VectorSearch(
    profiles=[
        VectorSearchProfile(
            name="profile",
            algorithm_configuration_name="hnsw-algorithm",
            vectorizer="azure-openai-vectorizer"
        )
    ],
    algorithms=[
        HnswAlgorithmConfiguration(name="hnsw-algorithm")
    ],
    vectorizers=[
        AzureOpenAIVectorizer(
                name="azure-openai-vectorizer",
                azure_open_ai_parameters=AzureOpenAIParameters(
                    resource_uri=azure_openai_endpoint,
                    deployment_id=azure_openai_embedding_deployment,
                    model_name = azure_openai_embedding_deployment,
                    api_key=azure_openai_key
                )
            )
    ]
)

index = SearchIndex(
    name = index_name,
    fields = fields,
    vector_search = vector_search)
result = index_client.create_or_update_index(index)

### Datasource 생성

Indexer에서 사용할 Datasoruce를 생성합니다.

이 예제에서는 Auzre Blob Storage를 Data Soruce로 사용 합니다.

#### Datasource로 사용 할 Blob Storage에 container 생성 후 샘플 파일을 업로드

In [ ]:
from azure.storage.blob import BlobServiceClient, BlobClient, ContainerClient

# Azure Blob Storage connection string
connection_string = os.getenv("AZURE_BLOB_CONNECTION_STRING")

# BlobServiceClient object 생성
blob_service_client = BlobServiceClient.from_connection_string(connection_string)

In [83]:
# 예제용 컨테이너(container) 생성 후 pdf 파일 업로드

# 컨테이너 생성
container_name = "skillset-container"
container_client = blob_service_client.get_container_client(container_name)
try:
    container_client.get_container_properties()
    print(f"{container_name}가 이미 존재합니다.")
except Exception as e:
    container_client.create_container()
    print(f"'{container_name}' 컨테이너를 생성하였습니다.")

# 파일 업로드
blob_name = "azure-search-partial.pdf"
blob_client = container_client.get_blob_client(blob_name)
local_file_path = f"data/{blob_name}"

# 파일 존재 시 삭제
try:
    blob_client.get_blob_properties()
    print(f"{blob_name} 파일이 이미 존재 합니다. 기존 파일을 삭제합니다.")
    blob_client.delete_blob()
    print(f"{blob_name} 파일이 삭제 되었습니다.")
except Exception as e:
    print(f"")

with open(local_file_path, "rb") as data:
    blob_client.upload_blob(data)

print(f"{blob_name} 파일이 {container_name} 컨테이너로 업로드 되었습니다.")

skillset-container가 이미 존재합니다.

azure-search-partial.pdf 파일이 skillset-container 컨테이너로 업로드 되었습니다.


#### Datasource 생성

In [78]:
indexer_client = SearchIndexerClient(endpoint=endpoint, credential=credential)

data_source_connection = SearchIndexerDataSourceConnection(
    name = search_datasource,
    type = "azureblob",
    connection_string = os.getenv("AZURE_BLOB_CONNECTION_STRING"),
    container = SearchIndexerDataContainer(name = container_name),
)

indexer_client.create_or_update_data_source_connection(data_source_connection)
print(f"'{search_datasource}' 데이터소스가 생성되었습니다.")

'aisearch-sample-datasource' 데이터소스가 생성되었습니다.


#### Skillset 생성

내장 된 Skillset을 이용하여 skillset을 생성합니다.

문서를 split 후 embedding 합니다.

In [72]:
skillset_name = "split-text-skilsets-test"

# skillset 정의
skillset = SearchIndexerSkillset(
    name=skillset_name,
    skills=[
        SplitSkill(
            name="Text split skill",
            default_language_code="ko",
            context="/document",
            inputs=[
                InputFieldMappingEntry(
                    name="text",
                    source="/document/content"
                )
            ],
            outputs=[
                OutputFieldMappingEntry(
                    name="textItems",
                    target_name="pages"
                )
            ],
            text_split_mode=TextSplitMode.Pages,
            maximum_page_length=2000,
            page_overlap_length=500,
        ),
        AzureOpenAIEmbeddingSkill(
            name="Embedding skill",
            resource_uri=azure_openai_endpoint,
            deployment_id=azure_openai_embedding_deployment,
            model_name=azure_openai_embedding_deployment,
            api_key=azure_openai_key,
            context="/document/pages/*",
            inputs=[
                InputFieldMappingEntry(
                    name="text",
                    source="/document/pages/*"
                )
            ],
            outputs=[
                OutputFieldMappingEntry(
                    name="embedding",
                    target_name="vector"
                )
            ]
        )
    ],
    index_projections=SearchIndexerIndexProjections(
        selectors=[
            SearchIndexerIndexProjectionSelector(
                target_index_name=index_name,
                parent_key_field_name="parent_id",
                source_context="/document/pages/*",
                mappings=[
                    InputFieldMappingEntry(
                        name="chunk",
                        source="/document/pages/*"
                    ),
                    InputFieldMappingEntry(
                        name="vector",
                        source="/document/pages/*/vector"
                    ),
                    InputFieldMappingEntry(
                        name="title",
                        source="/document/metadata_storage_name"
                    )
                ]
            )
        ],
        parameters=SearchIndexerIndexProjectionsParameters(projection_mode="skipIndexingParentDocuments")
    )
)

# Skillset 생성
indexer_client.create_or_update_skillset(skillset)
print(f"'{skillset_name}' skillset이 생성되었습니다.")

'split-text-skilsets-test' skillset이 생성되었습니다.


### Indexer 생성

이전에 생성한 data source, skillet을 이용하여 인덱서를 생성합니다.

인덱서가 실행되면, 데이터 소스로부터 데이터를 가져와 skillset으로 작업을 실행 후 대상 index에 데이터를 입력합니다.

원할 때 한 번 혹은 스케쥴링하여 실행 할 수 있습니다.

In [85]:
indexer_name = "split-text-skillsets-test-indexer"

indexer = SearchIndexer(
    name = indexer_name,
    data_source_name = search_datasource,
    target_index_name = index_name,
    skillset_name = skillset_name,
)

indexer_client.create_or_update_indexer(indexer)


In [79]:
# indexer 실행

indexer_client.run_indexer(indexer_name)

print("indexer 실행 중...")

indexer 실행 중...


인덱서 실행 후 Azure Portal에 접속하여 AI Search 서비스로 이동 후 Search management > Indexers 에서 생성 된 인덱서 확인 할 수 있습니다.

인덱서를 클릭하면 실행 된 내용을 확인 할 수 있습니다.

![Indexer](images/portal_ai_search_indexer_01.png)</p>
![Indexer detail](images/portal_ai_search_indexer_02.png)</p>
![Indexer index documents](images/portal_ai_search_indexer_03.png)